In [19]:
from nnsight.toolbox.UnifiedTransformer import UnifiedTransformer
from nnsight import LanguageModel

In [20]:
model: UnifiedTransformer = UnifiedTransformer.from_pretrained('EleutherAI/pythia-1B')
model = LanguageModel(model, tokenizer=model.tokenizer, device_map="cuda")

Loaded pretrained model EleutherAI/pythia-1B into HookedTransformer


In [22]:
def decoder(x):
  # decoder consists of final layer norm + unembedding
  return model.unembed(model.ln_final(x))

prompt = "John and Mary went to the store. John handed the milk to"

all_probs = []
with model.invoke(prompt) as invoker:
  for layer in model.blocks:
    logits = decoder(layer.output) # apply decoder to hidden state
    logits = logits[0,-1,:] # only over the final token
    probs = logits.softmax(dim=-1).save() # apply softmax to get probabilities and save
    all_probs.append(probs)
    # all_probs[i].value now stores the probability distribution over tokens after layer i

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [23]:
import plotly.express as px

mary_token = model.tokenizer(" Mary").input_ids[0] # token id for Mary

px.line(
    [layer_probs.value[mary_token].item() for layer_probs in all_probs],
    title="Probability of Mary after each layer, according to logit lens",
    labels={"value":"Probability", "index":"Layer"}
)